# Render Scene

Render scene normally without backward propgation.

## 0. Install and import requirements

* torch_psdr: core library
* pytorch (cuda): core dependency library
* pywavefront: read `.obj` format mesh object
* imageio: read image-based texture
* matplotlib: show rendered image in notebook

**⚠️ Please restart notebook kernel after installation!**

In [ ]:
# torch_psdr is in channel luling. This will install cuda based pytorch automatically
! conda install -y torch_psdr -c luling -c pytorch
# install other dependencies we need to run this example
! conda install -y imageio matplotlab -c conda-forge
# install forked pywavefront
! conda install pywavefront_uv -c luling

In [ ]:
%matplotlib inline

import torch_psdr as dr
import torch
from pywavefront_uv import Wavefront
import matplotlib.pyplot as plt

# define some global variables
device = "cuda:0"

## 1. Read mesh objects

In [ ]:
def load_obj(obj_path):
  scene = Wavefront(obj_path, create_materials=True, collect_faces=True)
  vertices = torch.tensor(scene.vertices, dtype=torch.float32, device=device)
  print(vertices.size())
  uvs = None if scene.parser.tex_coords == [] else torch.tensor(scene.parser.tex_coords, dtype=torch.float32, device=device)
  objs = {}
  for name, mesh in scene.meshes.items():
    if name is None:
      name = str(hash(mesh))
    indices = torch.tensor(mesh.faces, dtype=torch.int32, device=device)
    face_indices = indices[:,:,0]
    uv_indices = indices[:,:,1] if indices[0,0,1] != -1 else None
    material = mesh.materials[0]
    obj = None
    if name == "light":
      emit = torch.tensor(material.ambient[:3], dtype=torch.float32, device=device)
      obj = dr.AreaLight(vertices, face_indices, emit)
    else:
      diffuse = torch.tensor(material.diffuse[:3], dtype=torch.float32, device=device)
      normal = None
      roughness = torch.tensor([0.5], dtype=torch.float32, device=device)
      material = dr.DiffuseBsdfMaterial(diffuse, roughness, normal)
      obj = dr.Mesh(vertices, uvs, face_indices, uv_indices, material)
    objs[name] = obj
  return objs

In [ ]:
obj_path = "../data/input/cornell_box.obj"
objs = load_obj(obj_path)
print(f"object names: {objs.keys()}")

## 2. Set camera

In [ ]:
look_from = torch.tensor([278, 278, -800], dtype=torch.float32, device=device)
look_at = torch.tensor([278, 278, 0], dtype=torch.float32, device=device)
up = torch.tensor([0, 1, 0], dtype=torch.float32, device=device)
vfov = torch.tensor([torch.deg2rad(torch.tensor(38.0))], dtype=torch.float32, device=device)
height, width = 600, 600
camera = dr.PerspectiveCamera(
  look_from = look_from,
  look_at = look_at,
  up = up,
  vfov = vfov,
  height = height,
  width = width,
)

## 3. Construct scene

In [ ]:
cameras = [camera]
lights = [objs["light"]]
meshes = [obj for name, obj in objs.items() if name != "light"]
scene = dr.Scene(cameras, meshes, lights)

## 4. Configure renderer and render

In [ ]:
integrator = dr.PathIntegrator(
  n_pass = 1,
  spp_interior = 64,
  enable_light_visable=False,
  max_bounce=3,
  mis_light_samples=3,
  mis_bsdf_samples=3,
)

In [ ]:
imgs = integrator.renderC(scene)
img = imgs[0].detach().cpu().numpy()
plt.imshow(img)